In [1]:
import torch
import torch.nn  as nn 
import torch.optim  as optim
from torchvision import transforms, models
from PIL import Image 
import matplotlib.pyplot  as plt 
import os 
 

In [2]:
# # 配置参数
# CONTENT_IMG = "input.jpg"   # 当前目录下的输入图片
# OUTPUT_DIR = "."  # 输出目录 
# STYLE_IMG = "style.png"   # 风格图片
# IMAGE_SIZE = 512  # 图像尺寸
# NUM_STEPS = 5000  # 迭代次数
# STYLE_WEIGHT = 1  # 风格权重
# CONTENT_WEIGHT = 1  # 内容权重

In [3]:
# # 提取特征
# def get_features(image, model, layers):
#     features = {}
#     x = image
#     for idx, layer in enumerate(model):
#         x = layer(x)
#         if str(idx) in layers:
#             features[str(idx)] = x
#     return features
 
# # 计算Gram矩阵（风格特征）
# def gram_matrix(tensor):
#     _, c, h, w = tensor.size() 
#     tensor = tensor.view(c,  h * w)
#     return torch.mm(tensor,  tensor.t())

# # 图像预处理转换 
# preprocess = transforms.Compose([
#     transforms.Resize(IMAGE_SIZE),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])
 
# # 反标准化转换 
# def deprocess(tensor):
#     transform = transforms.Compose([
#         transforms.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
#         transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
#         transforms.ToPILImage()
#     ])
#     return transform(tensor)
 
# # 加载图像
# def load_image(image_path):
#     image = Image.open(image_path).convert('RGB') 
#     image = preprocess(image).unsqueeze(0)
#     return image.to(device,  torch.float) 
 
# # 保存图像
# def save_image(tensor, filename):
#     image = deprocess(tensor.squeeze(0).cpu().clone()) 
#     image.save(os.path.join(OUTPUT_DIR,  filename))
 

In [4]:
# # 创建输出目录
# os.makedirs(OUTPUT_DIR,  exist_ok=True)

# # 设备配置 
# device = torch.device("cuda"  if torch.cuda.is_available()  else "cpu")
# print(f"使用设备: {device}")
 
# # 加载轻量级SqueezeNet模型 [6]()
# model = models.squeezenet1_1(pretrained=True).features.to(device).eval() 
 
# # 冻结模型参数
# for param in model.parameters(): 
#     param.requires_grad  = False
 
# # 加载内容图像和风格图像 
# content_img = load_image(CONTENT_IMG)
# style_img = load_image(STYLE_IMG)
 
# # 创建目标图像（初始化为内容图像）
# input_img = content_img.clone().requires_grad_(True) 
 
# # 定义内容层和风格层
# content_layers = ['12']  # SqueezeNet中的关键特征层 
# style_layers = ['0', '3', '6', '8', '10']  # 多尺度风格特征层 

# # 优化器配置 
# optimizer = optim.LBFGS([input_img])
 
# # 训练循环 
# step = [0]
# while step[0] <= NUM_STEPS:
    
#     def closure():
#         # 梯度清零
#         optimizer.zero_grad() 
        
#         # 获取特征
#         content_features = get_features(content_img, model, content_layers)
#         style_features = get_features(style_img, model, style_layers)
#         input_features = get_features(input_img, model, set(content_layers + style_layers))
        
#         # 计算内容损失
#         content_loss = 0 
#         for layer in content_layers:
#             content_loss += torch.mean((input_features[layer]  - content_features[layer])**2)
        
#         # 计算风格损失
#         style_loss = 0 
#         for layer in style_layers:
#             input_gram = gram_matrix(input_features[layer])
#             style_gram = gram_matrix(style_features[layer])
#             norm_factor = input_features[layer].numel()  # 或 C * H * W
#             style_loss += torch.mean((input_gram  - style_gram)**2) / norm_factor
        
#         # 总损失
#         total_loss = CONTENT_WEIGHT * content_loss + STYLE_WEIGHT * style_loss
        
#         # 反向传播
#         total_loss.backward() 
        
#         # 每50步保存进度
#         if step[0] % 1000 == 0:
#             print(f"Step {step[0]}: Content Loss: {content_loss.item():.4f},  Style Loss: {style_loss.item():.4f}") 
#             save_image(input_img, f"output_step_{step[0]}.jpg")
        
#         step[0] += 1
#         return total_loss 
    
#     optimizer.step(closure) 
 
# # 保存最终结果
# save_image(input_img, "pic.jpg")

In [ ]:
# import cv2
# import dlib
# import numpy as np 
 
# # 初始化dlib人脸检测器和特征点预测器 
# detector = dlib.get_frontal_face_detector() 
# predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")   
 
# def get_landmarks(img):
#     """提取人脸68特征点+12个辅助点（图像四角、肩膀等）[11]()"""
#     gray = cv2.cvtColor(img,  cv2.COLOR_BGR2GRAY)
#     faces = detector(gray)
#     if not faces:
#         return None
#     shape = predictor(gray, faces[0])
#     points = [(p.x, p.y) for p in shape.parts()] 
    
#     # 添加12个辅助点（图像边界和身体关键点）
#     h, w = img.shape[:2] 
#     points.extend([(0,0),  (w,0), (0,h), (w,h), (0,h//2), (w,h//2), 
#                   (w//4,h), (3*w//4,h), (w//2,0), (w//2,h), (0,7*h//8), (w,7*h//8)])
#     return points
 
# def delaunay_triangulation(points, img_shape):
#     """Delaunay三角剖分[11]()"""
#     rect = (0, 0, img_shape[1], img_shape[0])
#     subdiv = cv2.Subdiv2D(rect)
#     for p in points:
#         subdiv.insert(p) 
#     triangles = subdiv.getTriangleList() 
#     return triangles.reshape(-1,  3, 2).astype(np.int32) 
 
# def warp_triangle(img1, img2, tri1, tri2):
#     """三角形仿射变换对齐[11]()"""
#     r1 = cv2.boundingRect(tri1) 
#     r2 = cv2.boundingRect(tri2) 
    
#     # 提取三角形区域并仿射变换
#     cropped1 = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]]
#     cropped2 = np.zeros_like(cropped1) 
    
#     offset1 = tri1 - r1[:2]
#     offset2 = tri2 - r2[:2]
    
#     mask = np.zeros((r1[3],  r1[2], 3), dtype=np.float32) 
#     cv2.fillConvexPoly(mask,  offset1, (1,1,1))
    
#     M = cv2.getAffineTransform(offset2.astype(np.float32),  offset1.astype(np.float32)) 
#     warped = cv2.warpAffine(img2[r2[1]:r2[1]+r2[3],  r2[0]:r2[0]+r2[2]], M, (r1[2], r1[3]))
    
#     # 融合三角形区域 
#     img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]] = img1[r1[1]:r1[1]+r1[3], r1[0]:r1[0]+r1[2]] * (1-mask) + warped * mask 
#     return img1
 
# def face_fusion(img1, img2, alpha=0.5):
#     """主融合函数"""
#     # 1. 获取特征点 
#     pts1 = get_landmarks(img1)
#     pts2 = get_landmarks(img2)
#     if pts1 is None:
#         raise ValueError("未检测到人脸1")
#     if pts2 is None:
#         raise ValueError("未检测到人脸2")
    
#     # 2. 三角剖分 
#     triangles = delaunay_triangulation(pts1, img1.shape) 
    
#     # 3. 创建空白画布
#     output = np.zeros_like(img1) 
    
#     # 4. 逐三角形仿射变换 
#     for tri in triangles:
#         idx = [np.where((pts1 == tuple(p)).all(axis=1))[0][0] for p in tri]
#         tri1 = np.array([pts1[i]  for i in idx])
#         tri2 = np.array([pts2[i]  for i in idx])
#         warp_triangle(output, img2, tri1, tri2)
    
#     # 5. 泊松融合消除边界[13]()
#     center = (img1.shape[1]//2,  img1.shape[0]//2) 
#     result = cv2.seamlessClone(output,  img1, np.ones_like(img1[:,:,0])*255,  center, cv2.NORMAL_CLONE)
    
#     # 6. 透明度混合 
#     return cv2.addWeighted(result,  alpha, img1, 1-alpha, 0)
 
# # 使用示例 
# img1 = cv2.imread("input.jpg") 
# img2 = cv2.imread("style.png") 
# result = face_fusion(img1, img2, alpha=0.7)
# cv2.imwrite("pic.jpg",  result)

ValueError: 未检测到人脸2

In [9]:
import cv2
import numpy as np 
from PIL import Image, ImageOps 
 
def preprocess_images(user_img_path, character_img_path):
    # 读取图像
    user_img_bgr = cv2.imread(user_img_path)
    char_img_bgr = cv2.imread(character_img_path)
    if user_img_bgr is None:
        raise ValueError(f"无法读取用户图片: {user_img_path}")
    if char_img_bgr is None:
        raise ValueError(f"无法读取角色图片: {character_img_path}")
    # 转为RGB
    user_img = cv2.cvtColor(user_img_bgr,  cv2.COLOR_BGR2RGB)
    char_img = cv2.cvtColor(char_img_bgr,  cv2.COLOR_BGR2RGB)
 
    # 人脸检测（使用OpenCV Haar级联）
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades  + 'haarcascade_frontalface_default.xml') 
    user_faces = face_cascade.detectMultiScale(user_img,  scaleFactor=1.1, minNeighbors=5)
    
    if len(user_faces) == 0:
        raise ValueError("未检测到用户人脸，请提供清晰正面照片")
        
    # 提取最大人脸区域 
    x, y, w, h = max(user_faces, key=lambda rect: rect[2]*rect[3])
    user_face = user_img[y:y+h, x:x+w]
    
    # 调整牛头人图像匹配人脸尺寸
    char_img = cv2.resize(char_img,  (w, h))
    return user_face, char_img 

def seamless_blending(user_face, char_img):
    # 创建掩码（全白矩形）
    mask = 126 * np.ones(char_img.shape,  char_img.dtype) 
    
    # 计算融合中心点（人脸区域中心）
    center = (user_face.shape[1]//2,  user_face.shape[0]//2) 
    
    # 泊松融合 
    blended = cv2.seamlessClone( 
        char_img, 
        user_face, 
        mask, 
        center, 
        cv2.NORMAL_CLONE 
    )
    return blended 

# 输入文件路径
user_img = "input.jpg" 
character_img = "style.png" 
 
try:
    # 预处理与对齐
    user_face, minotaur_resized = preprocess_images(user_img, character_img)
    
    # 融合处理 
    result = seamless_blending(user_face, minotaur_resized)
    
    # 保存结果
    Image.fromarray(result).save("pic.png") 
    print("融合完成！结果已保存为 pic.png") 
    
except Exception as e:
    print(f"处理失败: {str(e)}")

融合完成！结果已保存为 pic.png
